In [1]:
import sys
sys.path.append(r"D:\Papers and Journals\WAPORWA\modules")

import WA
from WA.pickle_basin import pickle_in, pickle_out

In [2]:
import sys
sys.path.insert(0, r"D:\Papers and Journals\WAPORWA\modules\WaPOR")

import WaPOR
WaPOR.list_available_mapsets()


WaPOR v3 module loaded
No API token required - uses public COG files
Use list_available_mapsets() to see all available datasets

Available WaPOR v3 Mapsets:
--------------------------------------------------------------------------------
L1-AETI-A            - Actual EvapoTranspiration and Interception (Global - Annual - 300m)
L1-AETI-D            - Actual EvapoTranspiration and Interception (Global - Dekadal - 300m)
L1-AETI-M            - Actual EvapoTranspiration and Interception (Global - Monthly - 300m)
L1-E-A               - Evaporation (Global - Annual - 300m)
L1-E-D               - Evaporation (Global - Dekadal - 300m)
L1-GBWP-A            - Gross biomass water productivity (Annual - 300m)
L1-I-A               - Interception (Global - Annual - 300m)
L1-I-D               - Interception (Global - Dekadal - 300m)
L1-NBWP-A            - Net biomass water productivity (Annual - 300m)
L1-NPP-D             - Net Primary Production (Global - Dekadal - 300m)
L1-NPP-M             - Net Pr

In [3]:
import os
import glob
#import WAPORWA modules
# os.chdir(r'D:\Papers and Journals\WAPORWA') #change to modules pat
import pandas as pd
from osgeo import gdal
import numpy as np
import osr
import calendar


Load your BASIN

In [6]:
import shapefile  # pyshp

# Path to your basin shapefile
BASIN_SHP = r"D:\Papers and Journals\WAPORWA\data\Litani\BasinLitani.shp"

sf = shapefile.Reader(BASIN_SHP)

xmin, ymin, xmax, ymax = sf.bbox
print("Basin BBOX:")
print("xmin =", xmin, "ymin =", ymin, "xmax =", xmax, "ymax =", ymax)


Basin BBOX:
xmin = 35.22916666666592 ymin = 33.0999999999998 xmax = 36.399999999999224 ymax = 34.04999999999972


In [7]:
# Example: define your basin bbox (latlim, lonlim)
latlim = [ymin - 0.25, ymax + 0.25]   # a bit bigger than basin
lonlim = [xmin - 0.25, xmax + 0.25]

# Output folder (like in 0_Collect_Prepare_data)
INPUT_FOLDER = r"D:\Papers and Journals\WAPORWA\data\Litani\Input"

WaPOR.PCP_monthly(
    Dir=INPUT_FOLDER,
    Startdate="2018-01-01",
    Enddate="2018-12-31",
    latlim=latlim,
    lonlim=lonlim,
    version=3,
    Waitbar=1
)



Date range: 2018-01-01 to 2018-12-31
Bounding box: lat [32.8499999999998, 34.29999999999972], lon [34.97916666666592, 36.649999999999224]
Found 12 rasters to download
Using scale factor: 0.1
[1/12] Downloading: L1-PCP-M.2018-01.tif
[2/12] Downloading: L1-PCP-M.2018-02.tif
[3/12] Downloading: L1-PCP-M.2018-03.tif
[4/12] Downloading: L1-PCP-M.2018-04.tif
[5/12] Downloading: L1-PCP-M.2018-05.tif
[6/12] Downloading: L1-PCP-M.2018-06.tif
[7/12] Downloading: L1-PCP-M.2018-07.tif
[8/12] Downloading: L1-PCP-M.2018-08.tif
[9/12] Downloading: L1-PCP-M.2018-09.tif
[10/12] Downloading: L1-PCP-M.2018-10.tif
[11/12] Downloading: L1-PCP-M.2018-11.tif
[12/12] Downloading: L1-PCP-M.2018-12.tif
Finished downloading L1-PCP-M


In [9]:
from WaPOR import GIS_functions as gis
import numpy as np
import os
import pandas as pd

v2_folder = r"D:\Papers and Journals\WAPORWA\data\Litani\InputOLD\L1_PCP_M2018"
v3_folder = r"D:\Papers and Journals\WAPORWA\data\Litani\Input\L1-PCP-M"

def load_array(tif):
    arr = gis.OpenAsArray(tif, nan_values=True)
    return arr.astype(float)

results = []

v2_files = sorted([f for f in os.listdir(v2_folder) if f.endswith(".tif")])
v3_files = sorted([f for f in os.listdir(v3_folder) if f.endswith(".tif")])

for f2, f3 in zip(v2_files, v3_files):
    month = f2[-6:-4]

    arr2 = load_array(os.path.join(v2_folder, f2))
    arr3 = load_array(os.path.join(v3_folder, f3))

    mean2 = np.nanmean(arr2)
    mean3 = np.nanmean(arr3)

    results.append([month, mean2, mean3, mean3 - mean2])

df = pd.DataFrame(results, columns=["Month", "PCP_v2", "PCP_v3", "Difference"])
print(df)


   Month      PCP_v2      PCP_v3  Difference
0     01  177.172394  219.662282   42.489887
1     02  132.274243  170.208363   37.934120
2     03   38.880714   37.318705   -1.562009
3     04   42.487980   61.197702   18.709723
4     05   31.523118   42.248695   10.725576
5     06    2.484412    3.116196    0.631784
6     07    0.012550    0.025705    0.013156
7     08    0.092470    0.078474   -0.013996
8     09    0.830251    2.639081    1.808830
9     10   36.733686   51.889552   15.155865
10    11   82.118760   83.940440    1.821681
11    12  159.972262  133.633127  -26.339135
